In [28]:
import numpy as np
from scipy import sparse


class BaseSmoothOracle:
    """
    Базовый класс для реализации оракулов.
    """
    def func(self, w):
        """
        Вычислить значение функции в точке w.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, w):
        """
        Вычислить значение градиента функции в точке w.
        """
        raise NotImplementedError('Grad oracle is not implemented.')

        
class BinaryLogistic(BaseSmoothOracle):
    """
    Оракул для задачи двухклассовой логистической регрессии.
    
    Оракул должен поддерживать l2 регуляризацию.
    """
    
    def __init__(self, l2_coef):
        """
        Задание параметров оракула.
        
        l2_coef - коэффициент l2 регуляризации
        """
        self.l2 = l2_coef / 2
     
    def func(self, X, y, w):
        """
        Вычислить значение функционала в точке w на выборке X с ответами y.
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
        
        w - одномерный numpy array
        
        """
        # логистическая функция потерь
        
        M = y * X.dot(w)
        
        loss = np.mean(np.log(1 + np.exp(-1 * M)))
        loss += self.l2  * np.dot(w, w)
        
        return loss
        
    def grad(self, X, y, w):
        """
        Вычислить градиент функционала в точке w на выборке X с ответами y.
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
        
        w - одномерный numpy array
        """
        
        #print(type(X), X.shape, X)
        #print(type(y), y.shape, y)
        #print(type(w), w.shape, w)
        
        M = y * X.dot(w)
        
        grad = 1 / (1 + np.exp(M))
        
        grad = np.mean(-1 * grad * y * X.T, axis=1) + 2 * self.l2 * w
        
        return grad

In [35]:
import numpy as np
from scipy import sparse
import time
import random

class GDClassifier:
    """
    Реализация метода градиентного спуска для произвольного
    оракула, соответствующего спецификации оракулов из модуля oracles.py
    """
    
    def __init__(self, loss_function, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, **kwargs):
        """
        loss_function - строка, отвечающая за функцию потерь классификатора. 
        Может принимать значения:
        - 'binary_logistic' - бинарная логистическая регрессия
                
        step_alpha - float, параметр выбора шага из текста задания
        
        step_beta- float, параметр выбора шага из текста задания
        
        tolerance - точность, по достижении которой, необходимо прекратить оптимизацию.
        Необходимо использовать критерий выхода по модулю разности соседних значений функции:
        если |f(x_{k+1}) - f(x_{k})| < tolerance: то выход 
        
        max_iter - максимальное число итераций     
        
        **kwargs - аргументы, необходимые для инициализации   
        """
        self.loss_function = loss_function
        self.step_alpha = step_alpha
        self.step_beta = step_beta
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.l2 = kwargs['l2_coef']
        
        self.oracle = BinaryLogistic
        
        
    def fit(self, X, y, w_0=None, trace=False):
        """
        Обучение метода по выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
        
        w_0 - начальное приближение в методе
        
        trace - переменная типа bool
      
        Если trace = True, то метод должен вернуть словарь history, содержащий информацию 
        о поведении метода. Длина словаря history = количество итераций + 1 (начальное приближение)
        
        history['time']: list of floats, содержит интервалы времени между двумя итерациями метода
        history['func']: list of floats, содержит значения функции на каждой итерации
        (0 для самой первой точки)
        """
        history = {}
        history['time'] = list()
        history['func'] = list()
        
        w_new = w_0
        w_old = w_0
        
        start = stop = 0
        
        history['func'].append(w_0)
        history['time'].append(0)
        
        for step in range(1, self.max_iter + 1):
            start = time.time()
            #print(X.shape, y.shape, w_old.shape)
            #print(self.oracle.grad(self, X, y, w_old))
            w_new = w_old - (self.step_alpha / step ** self.step_beta) * self.oracle.grad(self, X, y, w_old)
            history['func'].append(w_new)
            if np.abs(self.oracle.func(self, X, y, w_new) - self.oracle.func(self, X, y, w_old)) < self.tolerance:
                break
            w_old = w_new
            stop = time.time()
            history['time'].append(stop - start)
            
            
        self.w = w_new
        if trace:
            return history
        
    def predict(self, X):
        """
        Получение меток ответов на выборке X
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        return: одномерный numpy array с предсказаниями
        """
        proba = (1 + np.exp(-1 * X.dot(w))) ** -1
        answer = np.ones_like(proba)
        answer[proba < 0.5] = -1
        return answer

    def predict_proba(self, X):
        """
        Получение вероятностей принадлежности X к классу k
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        return: двумерной numpy array, [i, k] значение соответветствует вероятности
        принадлежности i-го объекта к классу k 
        
        """
        proba = (1 + np.exp(-1 * X.dot(w))) ** -1
        answer = np.empty(2 * len(proba)).reshape(-1, 2)
        answer[:,0] = proba
        answer[:,1] = 1 - proba
        return answer
        
    def get_objective(self, X, y):
        """
        Получение значения целевой функции на выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        y - одномерный numpy array
        
        return: float
        """
        return self.oracle.func(X, y, self.w)
        
    def get_gradient(self, X, y):
        """
        Получение значения градиента функции на выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        y - одномерный numpy array
        
        return: numpy array, размерность зависит от задачи
        """
        return self.oracle.grad(X, y, self.w)
    
    def get_weights(self):
        """
        Получение значения весов функционала
        """    
        return self.w



class SGDClassifier(GDClassifier):
    """
    Реализация метода стохастического градиентного спуска для произвольного
    оракула, соответствующего спецификации оракулов из модуля oracles.py
    """
    
    def __init__(self, loss_function, batch_size, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, random_seed=153, **kwargs):
        """
        loss_function - строка, отвечающая за функцию потерь классификатора. 
        Может принимать значения:
        - 'binary_logistic' - бинарная логистическая регрессия
        
        batch_size - размер подвыборки, по которой считается градиент
        
        step_alpha - float, параметр выбора шага из текста задания
        
        step_beta- float, параметр выбора шага из текста задания
        
        tolerance - точность, по достижении которой, необходимо прекратить оптимизацию
        Необходимо использовать критерий выхода по модулю разности соседних значений функции:
        если |f(x_{k+1}) - f(x_{k})| < tolerance: то выход 
        
        
        max_iter - максимальное число итераций (эпох)
        
        random_seed - в начале метода fit необходимо вызвать np.random.seed(random_seed).
        Этот параметр нужен для воспроизводимости результатов на разных машинах.
        
        **kwargs - аргументы, необходимые для инициализации
        """
        self.loss_function = loss_function
        self.batch_size = batch_size
        self.step_alpha = step_alpha
        self.step_beta = step_beta
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.seed = random_seed
        self.l2 = kwargs[0]
        
        self.oracle = BinaryLogostic
        
    def fit(self, X, y, w_0=None, trace=False, log_freq=1):
        """
        Обучение метода по выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
                
        w_0 - начальное приближение в методе
        
        Если trace = True, то метод должен вернуть словарь history, содержащий информацию 
        о поведении метода. Если обновлять history после каждой итерации, метод перестанет 
        превосходить в скорости метод GD. Поэтому, необходимо обновлять историю метода лишь
        после некоторого числа обработанных объектов в зависимости от приближённого номера эпохи.
        Приближённый номер эпохи:
            {количество объектов, обработанных методом SGD} / {количество объектов в выборке}
        
        log_freq - float от 0 до 1, параметр, отвечающий за частоту обновления. 
        Обновление должно проиходить каждый раз, когда разница между двумя значениями приближённого номера эпохи
        будет превосходить log_freq.
        
        history['epoch_num']: list of floats, в каждом элементе списка будет записан приближённый номер эпохи:
        history['time']: list of floats, содержит интервалы времени между двумя соседними замерами
        history['func']: list of floats, содержит значения функции после текущего приближённого номера эпохи
        history['weights_diff']: list of floats, содержит квадрат нормы разности векторов весов с соседних замеров
        (0 для самой первой точки)
        """
        random.seed(self.seed)
        history = {}
        history['time'] = list()
        history['func'] = list()
        
        w_new = w_0
        w_old = w_0
        
        start = stop = 0
        
        history['func'].append(w_0)
        history['time'].append(0)
        
        for step in range(1, self.max_iter + 1):
            start = time.time()
            ind = random.randint()
            w_new = w_old - (self.step_alpha / step ** self.step_beta) * self.oracle.grad( X, y, w_old)
            history['func'].append(w_new)
            if np.abs(self.oracle.func(X, y, w_new) - self.oracle.func(X, y, w_old)) < self.tolerance:
                break
            w_old = w_new
            stop = time.time()
            history['time'].append(stop - start)
            
            
        self.w = w_new
        if self.trace:
            return history

In [36]:
np.random.seed(10)
clf = GDClassifier(loss_function='binary_logistic', step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1)
l, d = 1000, 10
X = np.random.random((l, d))
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)

In [37]:

history = clf.fit(X, y, w_0=np.zeros(d), trace=True)
print(' '.join([str(x) for x in history['func']]))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [-0.00027869 -0.00711611 -0.01160488  0.00086325 -0.00926855 -0.00675723
 -0.00278599 -0.00450789 -0.01230398 -0.01148879] [ 0.0033826  -0.00851303 -0.01661495  0.0055345  -0.01241381 -0.00809236
 -0.00112568 -0.00418905 -0.01786489 -0.01644842] [ 0.00690655 -0.00888161 -0.01981649  0.00986344 -0.01414834 -0.00844196
  0.00082816 -0.00327148 -0.02149638 -0.01960599]
